# Lecture 11a: Fun with FFT and sound files

Based on: https://realpython.com/python-scipy-fft/

Define a function for generating pure sine wave tones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

SAMPLE_RATE = 44100  # Hertz
DURATION = 5  # Seconds

def generate_sine_wave(freq, sample_rate, duration):
    x = np.linspace(0, duration, sample_rate * duration, endpoint=False)
    frequencies = x * freq
    # 2pi because np.sin takes radians
    y = np.sin(2 * np.pi * frequencies)
    return x, y

# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, DURATION)
plt.plot(x, y)
plt.show()

Produce two tones, e.g. 400 Hz signal and a 4 kHz high-pitch noise

In [ ]:
_, nice_tone = generate_sine_wave(400, SAMPLE_RATE, DURATION)
_, noise_tone = generate_sine_wave(4000, SAMPLE_RATE, DURATION)
noise_tone = noise_tone * 0.3

mixed_tone = nice_tone + noise_tone
#mixed_tone = noise_tone

For the purposes of storing the tones in an audio file, the amplitude needs to be normalized to the range of 16-bit integer

In [ ]:
normalized_tone = np.int16((mixed_tone / mixed_tone.max()) * 32767)

plt.plot(normalized_tone[:1000])
plt.show()

Store the sound for playback

In [ ]:
from scipy.io import wavfile as wf

# Remember SAMPLE_RATE = 44100 Hz is our playback rate
wf.write("mysinewave.wav", SAMPLE_RATE, normalized_tone)


Can also try to record the sound (NB: won't work on datahub !)

In [ ]:
# import required libraries 
%pip install sounddevice
import sounddevice as sd 

print("Recording...")

# Start recorder with the given values 
# of duration and sample frequency 
recording = sd.rec(int(DURATION * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1) 

# Record audio for the given number of seconds 
sd.wait() 

print("Done")

# This will convert the NumPy array to an audio 
# file with the given sampling frequency 
wf.write("recording0.wav", SAMPLE_RATE, recording) 


### Fourier transforms

Now try to transform the time stream into frequency space using FFT

In [ ]:
from scipy.fft import fft, fftfreq

# Number of samples in normalized_tone
N = SAMPLE_RATE * DURATION

yf = fft(normalized_tone)
xf = fftfreq(N, 1 / SAMPLE_RATE)
print('Type of the output array: ',type(yf[0]))
print('Size of the input array: ',N)
print('Size of the Fourier transform: ',len(xf))
df = xf[1]-xf[0]
print(f'Width of the frequency bins: {df} Hz')

plt.plot(xf, np.abs(yf))
plt.xlabel('Frequency (Hz)')
plt.ylabel('FFT magnitude (a.u.)')
plt.show()


plt.figure()
plt.yscale('log')
plt.plot(xf, np.abs(yf))
plt.xlabel('Frequency (Hz)')
plt.ylabel('FFT magnitude (a.u.)')
plt.xlim(350,4050)
plt.show()

You notice that fft returns data for both positive and negative frequencies, produces the output array of the same size as input, and the output is a set of *complex* numbers. However, the information is reduntant: only half of the output values are unique. The magnitudes of the Fourier coefficients at negative frequencies are the same as at the corresponding positive frequencies. This is the property of the *real* Fourier transform, i.e. the transform applied to real-value signals. More precisely, $\mathrm{fft}(f)=\mathrm{fft}^*(-f)$

In [ ]:
print(xf[1],xf[-1])
print(yf[1],yf[-1])

We can use this fact to save computational time and storage by computing only half of the Fourier coefficients:

In [ ]:
from scipy.fft import rfft, rfftfreq

# Note the extra 'r' at the front
yf = rfft(normalized_tone)
xf = rfftfreq(N, 1 / SAMPLE_RATE)
print('Type of the output array: ',type(yf[0]))
print('Size of the input array: ',N)
print('Size of the Fourier transform: ',len(xf))
df = xf[1]-xf[0]
print(f'Width of the frequency bins: {df} Hz')

plt.plot(xf, np.abs(yf))
plt.xlim(1,5000)
plt.xlabel('Frequency (Hz)')
plt.ylabel('FFT magnitude (a.u.)')
plt.show()


Now let's look at the Fourier transform of a recorded sound:

In [ ]:
rate, data = wf.read("guitar.wav")
N=len(data)
print(rate, N)
time=np.arange(0, N)/rate

plt.plot(time, data)
plt.xlabel('time (sec)')
plt.ylabel('Sound a.u.)')
plt.show()

yf = rfft(data)
xf = rfftfreq(len(data), 1 / rate)
print('Type of the output array: ',type(yf[0]))
print('Size of the input array: ',len(data))
print('Size of the Fourier transform: ',len(xf))
df = xf[1]-xf[0]
print(f'Width of the frequency bins: {df} Hz')

mag = np.abs(yf)

plt.figure()
plt.loglog(xf, mag)
plt.xlabel('Frequency (Hz)')
plt.ylabel('FFT magnitude (a.u.)')
plt.show()

plt.figure()
plt.plot(xf, mag)
plt.yscale('log')
plt.xlim(100,2000)
plt.xlabel('Frequency (Hz)')
plt.ylabel('FFT magnitude (a.u.)')
plt.show()

fMax = xf[np.argmax(mag)]
print(f'Frequency of loudest tone: {fMax}')

See for example a chart of fundamental frequencies of notes <a href="https://pages.mtu.edu/~suits/notefreqs.html">here</a>